# Description

We predict drug-disease associations for all traits in using S-PrediXcan tissue-specific results and all drugs in LINCS consensus signatures. We use a method based on an [existing framework for drug-repositioning](https://www.nature.com/articles/nn.4618) to perform these predictions from genetic data. Instead of the Pearson/Spearman correlations, we use the dot product of disease z-scores (from S-PrediXcan) and drug z-scores from (LINCS). More details in the `drug_disease` module.

In this notebook we use the projection of S-PrediXcan results and LINCS data into the MultiPLIER latent space.

It saves results as HDF5 files with three keys: `full_prediction` (for all 4091 traits), `prediction` (only for traits and drugs present in the gold-standard/PharmacotheraphyDB) and `metadata`.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [3]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = True

In [4]:
PREDICTION_METHOD = "Module-based"

In [5]:
LINCS_DATA_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "lincs"
display(LINCS_DATA_DIR)
assert LINCS_DATA_DIR.exists()

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/lincs')

In [6]:
SPREDIXCAN_DATA_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "spredixcan" / "proj"
display(SPREDIXCAN_DATA_DIR)
assert SPREDIXCAN_DATA_DIR.exists()

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/spredixcan/proj')

In [7]:
OUTPUT_PREDICTIONS_DIR = Path(LINCS_DATA_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/lincs/predictions')

# Load PharmacotherapyDB gold standard

In [8]:
gold_standard = pd.read_pickle(
    Path(conf.RESULTS["DRUG_DISEASE_ANALYSES"], "gold_standard.pkl"),
)

In [9]:
display(gold_standard.shape)

(998, 3)

In [10]:
display(gold_standard.head())

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [11]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Projected data

In [12]:
input_file = Path(LINCS_DATA_DIR, "lincs-projection.pkl").resolve()

display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/lincs/lincs-projection.pkl')

In [13]:
lincs_projection = pd.read_pickle(input_file)

In [14]:
display(lincs_projection.shape)

(987, 1170)

In [15]:
display(lincs_projection.head())

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
LV1,0.036115,-0.358610,0.091067,0.008923,0.046469,0.136747,0.036151,0.034024,-0.027445,0.060644,...,0.069930,0.021537,0.020511,-0.006250,-0.002660,-0.017210,-0.004029,-0.145490,0.102504,0.057621
LV2,0.012281,0.009738,0.004650,-0.006554,0.021758,0.010425,0.000794,-0.000603,0.011534,-0.004425,...,0.000344,0.001123,-0.012675,-0.004849,0.000375,0.002623,-0.003252,-0.066209,0.025378,0.012453
LV3,-0.005533,-0.059174,0.013454,0.008906,-0.004939,0.033663,-0.010045,0.003824,0.009814,-0.007492,...,-0.021941,-0.002417,0.000155,0.002941,-0.015100,-0.016372,0.012249,0.009216,0.018254,-0.003516
LV4,-0.004151,-0.032884,0.005934,-0.003991,-0.028524,0.002709,0.007001,0.024704,0.005058,-0.004345,...,0.013806,0.003197,0.013099,0.002643,-0.008836,0.023852,0.028957,0.014681,0.000826,-0.009739
LV5,-0.015156,-0.005276,0.025747,0.000346,-0.013932,-0.010911,0.041403,-0.018285,0.003507,-0.007621,...,-0.011577,-0.015956,-0.027614,-0.001707,-0.000542,-0.016918,0.001959,0.024147,0.013004,-0.014166


# Load S-PrediXcan

In [16]:
phenomexcan_input_file_list = [
    f for f in SPREDIXCAN_DATA_DIR.glob("*.pkl") if f.name.startswith("spredixcan-")
]

In [17]:
display(len(phenomexcan_input_file_list))

49

In [18]:
pd.read_pickle(phenomexcan_input_file_list[0]).head()

,I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities,I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection,G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies,2395_4-Hairbalding_pattern_Pattern_4,20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet,4056_raw-Age_stroke_diagnosed,M13_FOREIGNBODY-Residual_foreign_body_in_soft_tissue,20002_1494-Noncancer_illness_code_selfreported_varicose_veins,I85-Diagnoses_main_ICD10_I85_Oesophageal_varices,6158_4-Why_reduced_smoking_Financial_reasons,...,41215_0-Detention_categories_Informal_not_formally_detained,M76-Diagnoses_main_ICD10_M76_Enthesopathies_of_lower_limb_excluding_foot,22601_35633208-Job_coding_vocational_or_industrial_trainerinstructor,H7_RETINALDETACH-Retinal_detachments_and_breaks,20003_1141172918-Treatmentmedication_code_celluvisc_1_singleuse_eye_drops,22601_35643209-Job_coding_careers_adviser_or_vocational_guidance_specialist_placement_advisor,20002_1537-Noncancer_illness_code_selfreported_joint_pain,20514-Recent_lack_of_interest_or_pleasure_in_doing_things,22601_53192534-Job_coding_builder_building_contractor,30280_raw-Immature_reticulocyte_fraction
LV1,-0.050381,-0.000908,-0.002332,0.057257,-0.000766,-0.023643,0.009266,-0.062103,-0.002072,-0.014975,...,0.020300,0.017245,-0.010104,0.013651,0.012818,-0.013683,-0.004481,0.026435,-0.024031,0.024488
LV2,-0.006339,-0.007228,-0.018861,-0.028251,0.005234,0.010514,-0.006057,-0.016552,0.006038,-0.024135,...,0.008744,-0.029665,0.003308,-0.005089,0.019235,-0.014829,-0.017671,-0.014115,0.012215,-0.023882
LV3,-0.024492,-0.003038,-0.006969,0.015853,-0.028497,0.000365,0.006933,-0.014768,0.035317,0.004242,...,0.024960,-0.009977,-0.024487,-0.007805,0.016711,-0.026119,-0.041412,0.028829,0.009171,-0.003566
LV4,-0.039315,0.013461,0.018323,0.017774,0.035927,0.019614,-0.018202,-0.029208,0.000738,-0.017938,...,-0.027179,-0.000500,-0.023355,0.005565,0.018736,-0.032961,0.010522,-0.007795,-0.035203,-0.016747
LV5,-0.006643,0.014785,-0.000057,0.013787,0.012504,-0.013875,0.018634,0.028472,0.045486,0.004247,...,-0.005076,0.011833,0.000759,0.014234,-0.020821,0.020727,0.035579,-0.064314,-0.029036,-0.026445


# Predict drug-disease associations

In [19]:
from drug_disease import predict_dotprod_neg

In [20]:
methods_to_run = [predict_dotprod_neg]

In [21]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)
    print(f"  shape: {phenomexcan_projection.shape}")

    for prediction_method in methods_to_run:
        # here I use different thresholds to select the LVs: the top 5, 10, 25 and 50, or all (None)
        for ntc in (None, 5, 10, 25, 50):
            prediction_method(
                lincs_projection,
                phenomexcan_input_file,
                phenomexcan_projection,
                OUTPUT_PREDICTIONS_DIR,
                PREDICTION_METHOD,
                doids_in_gold_standard,
                FORCE_RUN,
                n_top_conditions=ntc,
                use_abs=True,
            )

            print("\n")

spredixcan-mashr-zscores-Esophagus_Muscularis-projection.pkl
  shape: (987, 4091)
  predicting all_genes...
    shape: (1170, 4091)
  saving to: /home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/lincs/predictions/dotprod_neg/spredixcan-mashr-zscores-Esophagus_Muscularis-projection-all_genes-prediction_scores.h5
    saving full predictions...
    shape: (4786470, 3)


,score
count,4.786470e+06
mean,-1.238035e-21
std,2.583826e-02
min,-1.484040e+00
25%,-8.444432e-03
50%,5.534332e-06
75%,8.445070e-03
max,1.404311e+00


    key: full_prediction


/mnt/software/miniconda3/envs/phenoplier2/lib/python3.8/site-packages/tables/file.py:426: UserWarning: a closed node found in the registry: ``/full_prediction/meta/values_block_2/meta/_i_table``
  warnings.warn("a closed node found in the registry: "


    saving classifier data...
      shape (after DOID map): (1170, 351)
    shape: (410670, 3)


...

    key: full_prediction


/mnt/software/miniconda3/envs/phenoplier2/lib/python3.8/site-packages/tables/file.py:426: UserWarning: a closed node found in the registry: ``/full_prediction/meta/values_block_2/meta/_i_table``
  warnings.warn("a closed node found in the registry: "


    saving classifier data...
      shape (after DOID map): (1170, 351)
    shape: (410670, 3)


,score
count,410670.000000
mean,0.002011
std,0.013189
min,-0.509395
25%,-0.002697
50%,0.001585
75%,0.005887
max,0.543826


    key: prediction


